In [39]:
from __future__ import print_function
import sys
import os
import logging
import six
from radiomics import featureextractor, getFeatureClasses
import radiomics
%config Completer.use_jedi = False
import SimpleITK as sitk
import nibabel as nib
import glob
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import numpy as np
import logging
from utils_mask_sain import *
from utils import *
from utils_recaler import *


# Global: 3D

In [40]:
ls_image =sorted(glob.glob('./*_NAT*/*.nii'))

In [41]:
# Alternative: use hardcoded settings (separate for settings, input image types and enabled features)
settings = {}
settings['binWidth'] = 25
#settings['resampledPixelSpacing'] = [2,2]  # en 2D, on travaille dans un plan
settings['resampledPixelSpacing'] = [2, 2, 2]  # This is an example for defining resampling (voxels with size 3x3x3mm)
settings['interpolator'] = 'sitkBSpline'
settings['verbose'] = True
settings['force2D'] = True
settings['force2Ddimension'] = 0


extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.settings

{'minimumROIDimensions': 2,
 'minimumROISize': None,
 'normalize': False,
 'normalizeScale': 1,
 'removeOutliers': None,
 'resampledPixelSpacing': [2, 2, 2],
 'interpolator': 'sitkBSpline',
 'preCrop': False,
 'padDistance': 5,
 'distances': [1],
 'force2D': True,
 'force2Ddimension': 0,
 'resegmentRange': None,
 'label': 1,
 'additionalInfo': True,
 'binWidth': 25,
 'verbose': True}

In [42]:
# all_dat = pd.DataFrame()
# extractor.disableAllFeatures
# li_featureClass = ['firstorder', 'shape','glrlm', 'glszm', 'gldm', 'ngtdm']
# for featureClass in li_featureClass:
#     extractor.enableFeatureClassByName(featureClass)
# #On ne pourrait pas avoir les 2D...
# extractor.enableFeaturesByName(**{"glcm" : ['Autocorrelation',
#     'ClusterProminence',
#     'ClusterShade',
#     'ClusterTendency',
#     'Contrast',
#     'Correlation',
#     'DifferenceAverage',
#     'DifferenceEntropy',
#     'DifferenceVariance',
#     'Id',
#     'Idm',
#     'Idmn',
#     'Idn',
#     'Imc1',
#     'Imc2',
#     'InverseVariance',
#     'JointEnergy',
#     'JointEntropy',
#     'MCC',
#     'MaximumProbability',
#     'SumAverage',
#     'SumEntropy',
#     'SumSquares'
# ]})

# logger = logging.getLogger("radiomics.glcm")
# logger.setLevel(logging.ERROR) #Dégager les message inutiles (car on y a fait attention)

# #OUILLE LES PATIENTS S'APPELLENT ./DATASET et pas en slices???

# for imageName in tqdm.tqdm(ls_image):
#     maskName = imageName.replace('_NAT','').replace('.nii','_masked.nii')
#     classe_name = imageName.split('/')[-2].split('_')[1]
#     temps_inj = imageName.split('_')[-1].split('.')[0]

#     patient_num = imageName.split('/')[-1].split('_')[0]
#     #print(patient_num) 

        
#     mask_sitk = sitk.ReadImage(maskName) !=  0  ### ATTENTION C'est important ici on s'interesse au masque et non a l'image masquée
#     # print(imageName)
#     # print(mask_sitk.GetSize())
#     #print(maskName)
#     mr_sitk = sitk.ReadImage(imageName)
#     #print("nbe voxels :")
#     #print(sitk.GetArrayFromImage(mask_sitk).sum())


#     featureVector = extractor.execute(mr_sitk, mask_sitk)
#     df_idx = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVector.items() ])).iloc[0]
#     df_idx['classe_name'] = classe_name
#     df_idx['temps_inj'] = temps_inj
#     df_idx['patient_num'] = patient_num
#     df_idx = pd.DataFrame(df_idx).T
#     all_dat = pd.concat([df_idx,all_dat])
    


In [43]:
# all_dat.to_csv('./liver_tumors_all_LLB.csv')
# all_dat.to_excel('./global_excel.xlsx', index = False)


# Multislice

In [44]:
settings = {}
settings['binWidth'] = 25
settings['resampledPixelSpacing'] = [2,2]
#settings['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
settings['interpolator'] = 'sitkBSpline'
settings['verbose'] = True
settings['force2D'] = True
settings['force2Ddimension'] = 0   # ne pas oublier de le changer : fait: ce n'est plus True


extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor_sain = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.settings

{'minimumROIDimensions': 2,
 'minimumROISize': None,
 'normalize': False,
 'normalizeScale': 1,
 'removeOutliers': None,
 'resampledPixelSpacing': [2, 2],
 'interpolator': 'sitkBSpline',
 'preCrop': False,
 'padDistance': 5,
 'distances': [1],
 'force2D': True,
 'force2Ddimension': 0,
 'resegmentRange': None,
 'label': 1,
 'additionalInfo': True,
 'binWidth': 25,
 'verbose': True}

In [45]:
time_inj = ["_ART", "_PORT", "_VEIN","_TARD"]
#ls_image_prime = ls_image[(82 + 242):]
ls_image_prime = ls_image
ls_image_no_time = natsorted(list(set([eliminate_temps(x) for x in ls_image_prime])))
ls_image_full_times_surconfiance = [[name +time + ".nii" for time in time_inj] for name in ls_image_no_time]
ls_image_full_time = []
dict_image_full_time = {}
for li_names in ls_image_full_times_surconfiance:
    li_true_names = []
    for i,name in enumerate(li_names):
        if name in ls_image_prime:
            li_true_names.append(name)
    ls_image_full_time.append(li_true_names)
    classe_name = li_names[0].split('/')[-2].split('_')[1]
    patient_num = li_names[0].split('/')[-1].split('_')[0]
    dict_image_full_time[(patient_num, classe_name)] = li_true_names

ls_image_full_time = ls_image_full_time[0:10]

In [46]:
mode = "area"
fenetre_x = None
fenetre_y = None
fenetre_z  = range(-5,5)
label=extractor.settings.get('label')
label=1
minDims=2
extractor.disableAllFeatures()
extractor_sain.disableAllFeatures()
#utiliser des try et excepts ...
li_featureClass = ['firstorder','shape2D', 'glrlm', 'glszm', 'gldm', 'ngtdm']
li_featureClass_sain = ['firstorder', 'glrlm', 'glszm', 'gldm', 'ngtdm']
li_carac_glcm = ['Autocorrelation',
    'ClusterProminence',
    'ClusterShade',
    'ClusterTendency',
    'Contrast',
    'Correlation',
    'DifferenceAverage',
    'DifferenceEntropy',
    'DifferenceVariance',
    'Id',
    'Idm',
    'Idmn',
    'Idn',
    'Imc1',
    'Imc2',
    'InverseVariance',
    'JointEnergy',
    'JointEntropy',
    'MCC',
    'MaximumProbability',
    'SumAverage',
    'SumEntropy',
    'SumSquares'
]
for featureClass in li_featureClass:
    extractor.enableFeatureClassByName(featureClass)
for featureClass in li_featureClass_sain:
    extractor_sain.enableFeatureClassByName(featureClass)
extractor.enableFeaturesByName(**{"glcm": li_carac_glcm})
extractor_sain.enableFeaturesByName(**{"glcm": li_carac_glcm})
logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR) #Dégager les message inutiles (car on y a fait attention)
al_dat_slice = pd.DataFrame()

In [47]:
for num, li_image_names in tqdm.tqdm(enumerate(ls_image_full_time)):
    li_mask_names = [image_name.replace('_NAT','').replace('.nii','_masked.nii') for image_name in li_image_names]
    
    
    classe_name = li_image_names[0].split('/')[-2].split('_')[1]
    temps_inj = li_image_names[0].split('_')[-1].split('.')[0]
    patient_num = li_image_names[0].split('/')[-1].split('_')[0]
    
    li_images = [sitk.ReadImage(image_name) for image_name in li_image_names]
    li_mask = [sitk.ReadImage(mask_name) !=  0 for mask_name in li_mask_names]
    
    li_slices_masks, li_slices_images = equalize_slices(ls_image_full_time, dict_image_full_time,num = num, key = None, show = False)
    li_decal = np.array(find_best_decal_all_times_short(li_slices_masks, fenetre_z, fenetre_x=fenetre_x, fenetre_y=fenetre_y, mode=mode))
    nb_slices = len(li_slices_images[0])
    print("decalage", li_decal)

    for image_num_local, li_slices_image_unique in enumerate(li_slices_images):
        slice_depart = abs(np.min(li_decal[image_num_local],0))
        slice_fin = nb_slices - np.max(li_decal[image_num_local],0)
        for slice_num_ref in range(slice_depart, slice_fin):
            slice_num = slice_num_ref + li_decal[image_num_local]
            li_slices_mask_unique = li_slices_masks[image_num_local]
            slice_mask = li_slices_mask_unique[slice_num]   #format numpy
            slice_img = li_slices_image_unique[slice_num]   #format sitk
            slice_mask = sitk.GetImageFromArray(slice_mask) #passer en sitk
            slice_mask.CopyInformation(slice_img)
            
            lsif = sitk.LabelStatisticsImageFilter()
            lsif.Execute(slice_img, slice_mask)
            boundingBox = np.array(lsif.GetBoundingBox(label))
            ndims = np.sum((boundingBox[1::2] - boundingBox[0::2] + 1) > 3)  # UBound - LBound + 1 = Size
            if (sitk.GetArrayFromImage(slice_mask).sum() > 0) & (ndims >= minDims):
                slice_mask_sain = cercle_image_compos_connexe(slice_img,slice_mask) #on bosse au départ avec du sitk (pour toutes nos fonctions...)
                #On calcule le slice mask sain
                slice_mask_sain = sitk.GetImageFromArray(slice_mask_sain)
                slice_mask_sain.CopyInformation(slice_img)
                
                lsif = sitk.LabelStatisticsImageFilter()
                lsif.Execute(slice_img, slice_mask_sain)
                boundingBox = np.array(lsif.GetBoundingBox(label))
                ndims = np.sum((boundingBox[1::2] - boundingBox[0::2] + 1) > 3)  # UBound - LBound + 1 = Size
                imageName = li_image_names[image_num_local]
                if (sitk.GetArrayFromImage(slice_mask_sain).sum() > 0) & (ndims >= minDims):
                    try:
                        problem = False
                        #print("for extrac",ndims)
                        feature_vector_sain = extractor_sain.execute(slice_img, slice_mask_sain)
                        #print("extracted")
                        featureVector = extractor.execute(slice_img, slice_mask)
                        featureVector['slice_num'] = slice_num_ref #Tres important: on ne veut pas le slice_num mais le slice_num_ref
                        df_idx = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVector.items() ])).iloc[0]
                        df_idx_sain = pd.DataFrame(dict([ ("sain_" + k,pd.Series(v)) for k,v in feature_vector_sain.items() ])).iloc[0]
                        df_idx = pd.concat([df_idx,df_idx_sain])
                        df_idx['classe_name'] = classe_name
                        df_idx['temps_inj'] = temps_inj
                        df_idx['patient_num'] = patient_num
                        # al_dat_slice = al_dat_slice.append(df_idx)
                        df_idx = pd.DataFrame(df_idx).T
                        al_dat_slice = pd.concat([df_idx,al_dat_slice])
                    except Exception as e:
                        print(f'erreur pour slice {slice_num} pour image {imageName}')
                        print(e)
                        problem = True
                else:
                    problem = True
                    print(f'slice {slice_num} non valide sur zone saine pour image {imageName}')
                if problem:
                    feature_vector_sain = extractor_sain.execute(slice_img, slice_mask)
                    featureVector = extractor.execute(slice_img, slice_mask)
                    featureVector['slice_num'] = slice_num
                    df_idx = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVector.items() ])).iloc[0]
                    df_idx_sain = pd.DataFrame(dict([ ("sain_" + k,pd.Series(v)) for k,v in feature_vector_sain.items() ])).iloc[0]
                    df_idx_sain[:] = np.nan
                    df_idx = pd.concat([df_idx,df_idx_sain])
                    df_idx['classe_name'] = classe_name
                    df_idx['temps_inj'] = temps_inj
                    df_idx['patient_num'] = patient_num
                    # al_dat_slice = al_dat_slice.append(df_idx)
                    df_idx = pd.DataFrame(df_idx).T
                    al_dat_slice = pd.concat([df_idx,al_dat_slice])
        

10it [01:55, 11.60s/it]


In [48]:
print(extractor.enabledFeatures)
print(extractor_sain.enabledFeatures)

{'firstorder': [], 'shape2D': [], 'glrlm': [], 'glszm': [], 'gldm': [], 'ngtdm': [], 'glcm': ['Autocorrelation', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'Id', 'Idm', 'Idmn', 'Idn', 'Imc1', 'Imc2', 'InverseVariance', 'JointEnergy', 'JointEntropy', 'MCC', 'MaximumProbability', 'SumAverage', 'SumEntropy', 'SumSquares']}
{'firstorder': [], 'glrlm': [], 'glszm': [], 'gldm': [], 'ngtdm': [], 'glcm': ['Autocorrelation', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'Id', 'Idm', 'Idmn', 'Idn', 'Imc1', 'Imc2', 'InverseVariance', 'JointEnergy', 'JointEntropy', 'MCC', 'MaximumProbability', 'SumAverage', 'SumEntropy', 'SumSquares']}


In [49]:
al_dat_slice.to_csv('./liver_tumors_slices_LLB.csv')
al_dat_slice.to_excel('../multislice_excel_with_shape_2D_sain.xlsx', index = False)